In [23]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [24]:
%run ./seg_fraud.ipynb

3234
Return-Path: <james_ngola2002@maktoob.com>
X-Sieve: cmu-sieve 2.0
Return-Path: <james_ngola2002@maktoob.com>
Message-Id: <200210310241.g9V2fNm6028281@cs.CU>
From: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>
Reply-To: james_ngola2002@maktoob.com
To: webmaster@aclweb.org
Date: Thu, 31 Oct 2002 02:38:20 +0000
Subject: URGENT BUSINESS ASSISTANCE AND PARTNERSHIP
X-Mailer: Microsoft Outlook Express 5.00.2919.6900 DM
MIME-Version: 1.0
Content-Type: text/plain; charset="us-ascii"
Content-Transfer-Encoding: 8bit
X-MIME-Autoconverted: from quoted-printable to 8bit by sideshowmel.si.UM id g9V2foW24311
Status: O

FROM:MR. JAMES NGOLA.
CONFIDENTIAL TEL: 233-27-587908.
E-MAIL: (james_ngola2002@maktoob.com).

URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.


DEAR FRIEND,

I AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY GUARD ON 16TH JAN. 2001.


THE INCIDENT OCCURRED IN OUR PRESENCE WHILE WE WERE HOLDING MEETING 

In [25]:
%run ./seg_normal.ipynb

done.


In [26]:
%run ./fish_cleaner.ipynb

In [27]:
f1 = open('fraud.txt', 'r')
f2 = open('enron3.txt', 'r')

corpus_one = [item for item in f1]
corpus_two = [item for item in f2]

fs = split_set(corpus_one)
ns = split_enron_set(corpus_two)

fraud_set = []
norm_set = []

for i in range(len(fs)):
    fraud_set.append(super_clean(fs[i]))
for i in range(len(ns)):
    norm_set.append(super_clean(ns[i]))    

In [28]:
def bert_screen(x):        #remove emails from datasets that are less than 512 tokens long. 
    z = []
    for i in range(len(x)):
        temp = x[i].split()
        if len(temp) >= 512:
            z.append(x[i])
    return z

In [31]:
print('Length of fraud set before 512 token screening: {}'.format(len(fraud_set)))
print('Length of normal set before 512 token screening: {}'.format(len(norm_set)))

fraud_set = bert_screen(fraud_set)
norm_set = bert_screen(norm_set)

print('\n')
print('Length of fraud after before 512 token screening: {}'.format(len(fraud_set)))
print('Length of normal after before 512 token screening: {}'.format(len(norm_set)))

Length of fraud set before 512 token screening: 500
Length of normal set before 512 token screening: 500


Length of fraud after before 512 token screening: 500
Length of normal after before 512 token screening: 500


In [43]:
fraud_set = fraud_set[:500]
norm_set = norm_set[:500]

In [44]:
def label(fraud, norm):
    new_set = []
    for item in fraud:
        t = (item, 'fraud')
        new_set.append(t)
    for item in norm:
        t = (item, 'norm')
        new_set.append(t)
    return new_set    

In [45]:
x = label(fraud_set,norm_set)

In [46]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import torch.utils.data

In [47]:
g = pd.DataFrame(x)

In [48]:
pd.set_option('display.max_columns', None)
df = g
print(df.head())


                                                   0      1
0                  FROM:MR. JAMES NGOLA.    URGEN...  fraud
1               Dear Friend,  I am Mr. Ben Sulema...  fraud
2   FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...  fraud
3   FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...  fraud
4   Dear sir,    It is with a heart full of hope ...  fraud


In [49]:
from collections import Counter
print(Counter(df[1].values))

Counter({'fraud': 500, 'norm': 500})


In [50]:
df.dropna(inplace = True)


In [51]:
df = df.rename(columns={0: "text", 1: "type"})

In [52]:
print(len(df))
df.head()

1000


,text,type
0,FROM:MR. JAMES NGOLA. URGEN...,fraud
1,"Dear Friend, I am Mr. Ben Sulema...",fraud
2,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...,fraud
3,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...,fraud
4,"Dear sir, It is with a heart full of hope ...",fraud


In [53]:
df_fraud = df[df['type'] == 'fraud'] 
df_norm = df[df['type'] == 'norm'] 
df_norm = df_norm.sample(n=len(df_fraud))
df = df_norm.append(df_fraud)
df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

In [54]:
train_data = df.head(800)
test_data = df.tail(200)

In [55]:
x1 = np.array(train_data['text'])
x2 = np.array(train_data['type'])

y1 = np.array(test_data['text'])
y2 = np.array(test_data['type'])

In [56]:
t1 = []
t2 = []

for i in range(len(x1)):
    t1.append({'text': x1[i], 'type': x2[i]})
    
for i in range(len(y1)):
    t2.append({'text': y1[i], 'type': y2[i]})

In [57]:
train_data = t1
test_data = t2

In [58]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

In [59]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

Cell above tokenizes text sequences of up to 512 tokens, as that is the max input size of BERT.
Datapoints posessing less than 512 tokens are 



In [60]:
train_y = []
test_y = []
t1 = []
t2 = []

for item in train_labels:
    if str(item) == 'fraud': t1.append(1)
    else: t1.append(0)    

        
for item in test_labels:
    if str(item) == 'fraud': t2.append(1)
    else: t2.append(0)        
    
train_y = np.array(t1)
test_y = np.array(t2)

In [61]:
print("TRAINING DAtA LABEL VECTOR")
print(train_y)
print(test_y)

TRAINING DAtA LABEL VECTOR
[0 0 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 1 1
 0 1 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0
 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0
 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1
 0 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0
 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1
 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1
 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1
 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 0 1
 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 

In [62]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__() #DEFAULT CONSTRUCTOR INIT
    
        self.bert = BertModel.from_pretrained('bert-base-uncased') #USE BERT BASE FOR LIGHTER LOAD
        self.dropout = nn.Dropout(dropout) #DROPOUT = DEFAULT
        self.linear = nn.Linear(768, 1)    #LINEAR ACTIVATION LAYER, HIDDEN VECTOR LENGTH 768
        self.sigmoid = nn.Sigmoid()        #SIGMOID ACTIVATION LAYER, S SHAPE DECISION BOUNDARY
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
    #FUNCTION ABOVE DEFINES FLOW OF DATA FOR BERT MODEL.
    #POOLED OUTPUT = DEFAULT BERT POOLED OUTPUT, ABSTRACTION OF DATAPOINT
    #DROPOUT = DEFAULT BERT DROPOUT
    #LINEAR_OUTPUT = LINEAR ACTIVATION WITH HIDDEN SIZE 768
    #FINAL STEP, PUT RESULT INTO SIGMOID ACTIVATION FOR RESULT

In [63]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

#CREATE TENSORS FOR TRAINING. 

In [64]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [65]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=1)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=1)

STATEMENT BELOW ITERATES THROUGH EACH ITEM IN THE TRAINING DATASET AT ONE EPOCH

In [66]:
BATCH_SIZE = 1
EPOCHS = 1
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  1
0/800.0 loss: 0.5651776790618896 
Epoch:  1
1/800.0 loss: 0.6311807930469513 
Epoch:  1
2/800.0 loss: 0.5909890929857889 
Epoch:  1
3/800.0 loss: 0.6638465076684952 
Epoch:  1
4/800.0 loss: 0.714335811138153 
Epoch:  1
5/800.0 loss: 0.7297720909118652 
Epoch:  1
6/800.0 loss: 0.7072210567338126 
Epoch:  1
7/800.0 loss: 0.7209253460168839 
Epoch:  1
8/800.0 loss: 0.7008258832825555 
Epoch:  1
9/800.0 loss: 0.6895004570484161 
Epoch:  1
10/800.0 loss: 0.6720695116303184 
Epoch:  1
11/800.0 loss: 0.6847949773073196 
Epoch:  1
12/800.0 loss: 0.6731671049044683 
Epoch:  1
13/800.0 loss: 0.6827818666185651 
Epoch:  1
14/800.0 loss: 0.6739884535471599 
Epoch:  1
15/800.0 loss: 0.6799650862812996 
Epoch:  1
16/800.0 loss: 0.6886548014248118 
Epoch:  1
17/800.0 loss: 0.7010260588592954 
Epoch:  1
18/800.0 loss: 0.7013108573461834 
Epoch:  1
19/800.0 loss: 0.7033329159021378 
Epoch:  1
20/800.0 loss: 0.7100825196220761 
Epoch:  1
21/800.0 loss: 0.7068696238777854 
Epoch:  1
22/800.0 lo

Epoch:  1
182/800.0 loss: 0.5311396026709041 
Epoch:  1
183/800.0 loss: 0.5300990309890198 
Epoch:  1
184/800.0 loss: 0.5284550044182185 
Epoch:  1
185/800.0 loss: 0.5267097343520452 
Epoch:  1
186/800.0 loss: 0.5259555829240676 
Epoch:  1
187/800.0 loss: 0.5253048394113145 
Epoch:  1
188/800.0 loss: 0.5245268609000262 
Epoch:  1
189/800.0 loss: 0.523483676894715 
Epoch:  1
190/800.0 loss: 0.521571415335096 
Epoch:  1
191/800.0 loss: 0.5210151555171857 
Epoch:  1
192/800.0 loss: 0.5201916715487297 
Epoch:  1
193/800.0 loss: 0.5185127943446955 
Epoch:  1
194/800.0 loss: 0.5172101552669819 
Epoch:  1
195/800.0 loss: 0.5163006166718445 
Epoch:  1
196/800.0 loss: 0.5154228608317787 
Epoch:  1
197/800.0 loss: 0.5150930723457625 
Epoch:  1
198/800.0 loss: 0.5133589774220433 
Epoch:  1
199/800.0 loss: 0.5121009455621243 
Epoch:  1
200/800.0 loss: 0.510505895709517 
Epoch:  1
201/800.0 loss: 0.5095189656361495 
Epoch:  1
202/800.0 loss: 0.5086823255557732 
Epoch:  1
203/800.0 loss: 0.507749972

Epoch:  1
359/800.0 loss: 0.37012427219500144 
Epoch:  1
360/800.0 loss: 0.36945133268255276 
Epoch:  1
361/800.0 loss: 0.36868361062601784 
Epoch:  1
362/800.0 loss: 0.3678928053740299 
Epoch:  1
363/800.0 loss: 0.367072213359259 
Epoch:  1
364/800.0 loss: 0.36627642073451655 
Epoch:  1
365/800.0 loss: 0.36561136816938716 
Epoch:  1
366/800.0 loss: 0.36489207405160495 
Epoch:  1
367/800.0 loss: 0.3642006463209248 
Epoch:  1
368/800.0 loss: 0.3634364438775755 
Epoch:  1
369/800.0 loss: 0.3626897105695428 
Epoch:  1
370/800.0 loss: 0.3620248606825132 
Epoch:  1
371/800.0 loss: 0.36133277354141075 
Epoch:  1
372/800.0 loss: 0.3605846480613739 
Epoch:  1
373/800.0 loss: 0.3598412141243723 
Epoch:  1
374/800.0 loss: 0.35906537433465324 
Epoch:  1
375/800.0 loss: 0.3583896152001429 
Epoch:  1
376/800.0 loss: 0.35770695020175425 
Epoch:  1
377/800.0 loss: 0.35696319910505464 
Epoch:  1
378/800.0 loss: 0.35620460181563385 
Epoch:  1
379/800.0 loss: 0.3555578691982909 
Epoch:  1
380/800.0 loss

Epoch:  1
536/800.0 loss: 0.2839167351163299 
Epoch:  1
537/800.0 loss: 0.28353934203271086 
Epoch:  1
538/800.0 loss: 0.28310098727829824 
Epoch:  1
539/800.0 loss: 0.2826654157221869 
Epoch:  1
540/800.0 loss: 0.28223857824826637 
Epoch:  1
541/800.0 loss: 0.2818134607080494 
Epoch:  1
542/800.0 loss: 0.281420288953937 
Epoch:  1
543/800.0 loss: 0.2810260648770696 
Epoch:  1
544/800.0 loss: 0.280630404720886 
Epoch:  1
545/800.0 loss: 0.28021624732776224 
Epoch:  1
546/800.0 loss: 0.2798180010108451 
Epoch:  1
547/800.0 loss: 0.279422997531012 
Epoch:  1
548/800.0 loss: 0.27907088792497775 
Epoch:  1
549/800.0 loss: 0.27866587459363723 
Epoch:  1
550/800.0 loss: 0.27831451613084157 
Epoch:  1
551/800.0 loss: 0.2779061832006319 
Epoch:  1
552/800.0 loss: 0.2775710156056053 
Epoch:  1
553/800.0 loss: 0.2771576190105091 
Epoch:  1
554/800.0 loss: 0.2767575132551494 
Epoch:  1
555/800.0 loss: 0.2763274118987669 
Epoch:  1
556/800.0 loss: 0.27592311439704637 
Epoch:  1
557/800.0 loss: 0.2

Epoch:  1
713/800.0 loss: 0.22365332990415207 
Epoch:  1
714/800.0 loss: 0.22338645973770352 
Epoch:  1
715/800.0 loss: 0.22314250850841902 
Epoch:  1
716/800.0 loss: 0.22285803377555588 
Epoch:  1
717/800.0 loss: 0.22258887244103098 
Epoch:  1
718/800.0 loss: 0.22231962392948598 
Epoch:  1
719/800.0 loss: 0.22204648250869166 
Epoch:  1
720/800.0 loss: 0.2217647583248422 
Epoch:  1
721/800.0 loss: 0.22150457197145 
Epoch:  1
722/800.0 loss: 0.2212341616849929 
Epoch:  1
723/800.0 loss: 0.22096639398732404 
Epoch:  1
724/800.0 loss: 0.22070505845135657 
Epoch:  1
725/800.0 loss: 0.22043047024876125 
Epoch:  1
726/800.0 loss: 0.22016046559271574 
Epoch:  1
727/800.0 loss: 0.21988920802199333 
Epoch:  1
728/800.0 loss: 0.21963540454163882 
Epoch:  1
729/800.0 loss: 0.21938590278517303 
Epoch:  1
730/800.0 loss: 0.2191131875484611 
Epoch:  1
731/800.0 loss: 0.21884980028026987 
Epoch:  1
732/800.0 loss: 0.2185807795597985 
Epoch:  1
733/800.0 loss: 0.2183314608105481 
Epoch:  1
734/800.0 l

In [67]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss() #binomial cross entropy loss
        loss = loss_func(logits, labels) 
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       117
           1       1.00      1.00      1.00        83

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



PRECISION = TRUE POS / TRUE POS + FALSE POS
RECALL = TRUE POS / TRUE POS + FALSE NEG
F1 = (2 * RECALL * PRECISION) / ( RECALL + PRECISION)